In [1]:
import pickle
import numpy as np
import pandas as pd
from datetime import datetime

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
import keras_tuner as kt
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping

from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report

In [2]:
with open(r'.\Reviews_Dataset_Splits\X_train.pkl', 'rb') as f:
    X_train = pickle.load(f)

with open(r'.\Reviews_Dataset_Splits\X_val.pkl', 'rb') as f:
    X_val = pickle.load(f)

with open(r'.\Reviews_Dataset_Splits\X_test.pkl', 'rb') as f:
    X_test = pickle.load(f)

with open(r'.\Reviews_Dataset_Splits\y_train.pkl', 'rb') as f:
    y_train = pickle.load(f)

with open(r'.\Reviews_Dataset_Splits\y_val.pkl', 'rb') as f:
    y_val = pickle.load(f)

with open(r'.\Reviews_Dataset_Splits\y_test.pkl', 'rb') as f:
    y_test = pickle.load(f)

y_train = keras.utils.to_categorical(y_train-1, num_classes=5)
y_val = keras.utils.to_categorical(y_val-1, num_classes=5)
y_test = keras.utils.to_categorical(y_test-1, num_classes=5)

with open(r'.\embeddingMatrix_Reviews.pkl', 'rb') as f:
    embedding_matrix = pickle.load(f)

num_tokens = len(embedding_matrix) # total vocabulary +1 or length of embedding matrix
embedding_dim = 300 # dimension of the vector of a single word
MAX_REVIEW_LEN = 250 # maximum words in a review
num_classes = 5

In [22]:
def build_model(hp, min_layers, max_layers, test_optimizers, test_activations, use_Dropout,
                units_min_value, units_max_value, units_step):
    embedding_layer = keras.layers.Embedding(
        num_tokens,
        embedding_dim,
        embeddings_initializer=keras.initializers.Constant(embedding_matrix),
        input_length=MAX_REVIEW_LEN,
        trainable=True,
        mask_zero=True)
   
    model = keras.Sequential()
    model.add(embedding_layer)
    
    num_layers = hp.Int('num_layers', min_layers, max_layers)

    if test_activations:
            activation = hp.Choice(f'activation', ['softplus', 'softsign', 'relu', 'tanh'])
    else:
        activation = 'tanh'  # Default activation

    for i in range(num_layers):
        units = hp.Int(f'units_{i}', min_value=units_min_value, max_value=units_max_value, step=units_step)

        if i < num_layers - 1:  # For all layers except the last one
            model.add(keras.layers.LSTM(units, activation=activation, return_sequences=True))
        else:  # For the last LSTM layer
            model.add(keras.layers.LSTM(units, activation=activation))

        if use_Dropout:
            model.add(layers.Dropout(rate=hp.Choice(f'dropout_rate_{i}', [0.0, 0.2])))

    model.add(layers.Dense(num_classes, activation='softmax'))
    
    # Optimizer
    if test_optimizers:
        optimizer = hp.Choice('optimizer', ['SGD', 'RMSprop', 'Adam', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam'])
    else:
        optimizer = 'adam'
    
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [27]:
NUM_EPOCHS = 12
BATCH_SIZE = 128

now = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
directory = f'KerasTuner_Logs/RNN/RNN_V1_3Layer_Density_Optimization_{now}'

# Callbacks
tensorboard = TensorBoard(log_dir=f'TensorBoard_Logs/RNN/RNN_V1_3Layer_Density_Optimization_{now}')
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True, verbose=1)

tuner = kt.GridSearch(
    lambda hp: build_model(hp, min_layers=3, max_layers=3, test_optimizers=False, test_activations=False, use_Dropout=False, 
                           units_min_value=64, units_max_value=128, units_step=32),
    objective=kt.Objective("val_loss", direction="min"),
    max_trials=None,
    executions_per_trial=1,
    directory=directory,
    project_name='Reviews_Classification')

In [28]:
tuner.search(x=X_train,
             y=y_train,
             verbose=1,
             epochs=NUM_EPOCHS,
             batch_size=BATCH_SIZE,
             callbacks=[tensorboard, early_stopping],
             validation_data=(X_val, y_val))

Trial 27 Complete [00h 03m 48s]
val_loss: 1.0865203142166138

Best val_loss So Far: 1.0782015323638916
Total elapsed time: 01h 40m 21s


In [29]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 3, 'units_0': 64, 'units_1': 128, 'units_2': 64}

In [26]:
tuner.get_best_models()[0].summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 250, 300)          16058400  
                                                                 
 lstm (LSTM)                 (None, 250, 128)          219648    
                                                                 
 lstm_1 (LSTM)               (None, 96)                86400     
                                                                 
 dense (Dense)               (None, 5)                 485       
                                                                 
Total params: 16,364,933
Trainable params: 16,364,933
Non-trainable params: 0
_________________________________________________________________


In [21]:
tuner.results_summary()

Results summary
Results in KerasTuner_Logs/RNN/RNN_V1_2Layer_Density_Optimization_2024-07-16_15-45-03\Reviews_Classification
Showing 10 best trials
Objective(name="val_loss", direction="min")

Trial 0000 summary
Hyperparameters:
num_layers: 2
units_{i}: 64
Score: 1.0833885669708252

Trial 0002 summary
Hyperparameters:
num_layers: 2
units_{i}: 128
Score: 1.0934594869613647

Trial 0001 summary
Hyperparameters:
num_layers: 2
units_{i}: 96
Score: 1.0941708087921143
